## Enable Resource Providers

There are some resource providers that must be registered in your Azure subscription. Follow these steps to ensure that they're registered.

Sign into the Azure portal at https://portal.azure.com using the Microsoft credentials associated with your subscription.
On the Home page, select Subscriptions (or expand the ≡ menu, select All Services, and in the General category, select Subscriptions).
Select your Azure subscription (if you have multiple subscriptions, select the one you created by redeeming your Azure Pass).
In the blade for your subscription, in the pane on the left, in the Settings section, select Resource providers.
In the list of resource providers, ensure the following providers are registered (if not, select them and click register):

* Microsoft.Web
* Microsoft.ManagedIdentity
* Microsoft.Search
* Microsoft.Storage
* Microsoft.CognitiveServices
* Microsoft.AlertsManagement
* microsoft.insights
* Microsoft.KeyVault
* Microsoft.ContainerInstance

## Provision an Azure AI Services resource

Azure AI Services are cloud-based services that encapsulate artificial intelligence capabilities you can incorporate into your applications. You can provision individual Azure AI services resources for specific APIs (for example, **Language** or **Vision**), or you can provision a single **Azure AI Services** resource that provides access to multiple Azure AI services APIs through a single endpoint and key. In this case, you'll use a single **Azure AI Services** resource.

1. Open the Azure portal at `https://portal.azure.com`, and sign in using the Microsoft account associated with your Azure subscription.
2. In the top search bar, search for *Azure AI services*, select **Azure AI Services**, and create an Azure AI services multi-service account resource with the following settings:
    - **Subscription**: *Your Azure subscription*
    - **Resource group**: *Choose or create a resource group (if you are using a restricted subscription, you may not have permission to create a new resource group - use the one provided)*
    - **Region**: *Choose any available region*
    - **Name**: *Enter a unique name*
    - **Pricing tier**: Standard S0
3. Select the required checkboxes and create the resource.
4. Wait for deployment to complete, and then view the deployment details.
5. Go to the resource and view its **Keys and Endpoint** page. This page contains the information that you will need to connect to your resource and use it from applications you develop. Specifically:
    - An HTTP *endpoint* to which client applications can send requests.
    - Two *keys* that can be used for authentication (client applications can use either key to authenticate).
    - The *location* where the resource is hosted. This is required for requests to some (but not all) APIs.

## Use a REST Interface

The Azure AI services APIs are REST-based, so you can consume them by submitting JSON requests over HTTP. In this example, you'll explore a console application that uses the **Language** REST API to perform language detection; but the basic principle is the same for all of the APIs supported by the Azure AI Services resource.


In [ ]:
COG_SERVICE_ENDPOINT="your_cognitive_services_endpoint"
COG_SERVICE_KEY="your_cognitive_services_key"

In [24]:
COG_SERVICE_ENDPOINT="https://testaianshuooo7.cognitiveservices.azure.com/"
COG_SERVICE_KEY="a2c143d9d33e404488839fe03c5539c6"

In [10]:
import os
import http.client, base64, json, urllib
from urllib import request, parse, error


In [11]:
text = input ("Enter some text")
jsonBody = {
            "documents":[
                {"id": 1,
                 "text": text}
            ]
        }

# Let's take a look at the JSON we'll send to the service
print(json.dumps(jsonBody, indent=2))

{
  "documents": [
    {
      "id": 1,
      "text": "test I love poland"
    }
  ]
}


In [23]:
# Make an HTTP request to the REST interface
uri = COG_SERVICE_ENDPOINT.rstrip('/').replace('https://', '')
conn = http.client.HTTPSConnection(uri)

# Add the authentication key to the request header
headers = {
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': COG_SERVICE_KEY
}

# Use the Text Analytics language API
conn.request("POST", "/text/analytics/v3.1/languages?", str(jsonBody).encode('utf-8'), headers)

# Send the request
response = conn.getresponse()
data = response.read().decode("UTF-8")

print(data)

{"error":{"code":"401","message":"Access denied due to invalid subscription key or wrong API endpoint. Make sure to provide a valid key for an active subscription and use a correct regional API endpoint for your resource."}}


In [14]:
uri

'testaianshuooo7.cognitiveservices.azure.com'

In [ ]:
# If the call was successful, get the response
if response.status == 200:

    # Display the JSON response in full (just so we can see it)
    results = json.loads(data)
    print(json.dumps(results, indent=2))

    # Extract the detected language name for each document
    for document in results["documents"]:
        print("\nLanguage:", document["detectedLanguage"]["name"])

## Use an SDK

You can write code that consumes Azure AI services REST APIs directly, but there are software development kits (SDKs) for many popular programming languages, including Microsoft C#, Python, and Node.js. Using an SDK can greatly simplify development of applications that consume Azure AI services.



In [20]:
!pip install azure-ai-textanalytics==5.3.0 --quiet

In [21]:
import os
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

In [26]:
# Create client using endpoint and key
credential = AzureKeyCredential(COG_SERVICE_KEY)
client = TextAnalyticsClient(endpoint=COG_SERVICE_ENDPOINT, credential=credential)

# Call the service to get the detected language
detectedLanguage = client.detect_language(documents = [text])[0]

ClientAuthenticationError: (401) Access denied due to invalid subscription key or wrong API endpoint. Make sure to provide a valid key for an active subscription and use a correct regional API endpoint for your resource.
Code: 401
Message: Access denied due to invalid subscription key or wrong API endpoint. Make sure to provide a valid key for an active subscription and use a correct regional API endpoint for your resource.

# Analyze Text